In [1]:
# import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, re, time, math, tqdm, itertools
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import plotly.express as px
import plotly.offline as pyo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
import keras
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, BatchNormalization, Dense
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping

In [2]:
# import glob

# path = r'../input/ids-intrusion-csv'                  
# all_files = glob.glob(os.path.join(path, "*.csv"))

# print(all_files)

In [3]:
# all_files.pop(6)

In [4]:
# Memory is not enough to read all csv files at the same time because we didn't upgrate to pro version

#　df_from_each_file = (pd.read_csv(f) for f in all_files)
# df_list = []

# for f in all_files:
#     df_csv = pd.read_csv(f)
#     df_list.append(df_csv)

# del df_csv

# df = pd.concat(df_list, axis=0, ignore_index=True)

In [5]:
df_list = []
df1 = pd.read_csv('../input/ids-intrusion-csv/02-14-2018.csv', index_col=None, header=0)
df2 = pd.read_csv('../input/ids-intrusion-csv/02-15-2018.csv', index_col=None, header=0)
df3 = pd.read_csv('../input/ids-intrusion-csv/03-01-2018.csv', index_col=None, header=0)
df4 = pd.read_csv('../input/ids-intrusion-csv/03-02-2018.csv', index_col=None, header=0)

df_list.append(df1)
df_list.append(df2)
df_list.append(df3)
df_list.append(df4)


del df1, df2, df3, df4
df = pd.concat(df_list, axis=0, ignore_index=True)

FileNotFoundError: ignored

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# check the number of values for Protocol
df["Protocol"].value_counts()

In [ ]:
# check the number of values for Label
df["Label"].value_counts()

In [ ]:
label_del = df[df["Label"].isin(['Label'])]
label_del['Label']

In [ ]:
index = label_del['Label'].index
print(index)

In [ ]:
for i in index:
    df = df.drop([i])

In [ ]:
df["Label"].value_counts()

In [ ]:
# make a plot number of labels
sns.set(rc={'figure.figsize':(24, 6)})
plt.xlabel('Attack Type')
sns.set_theme()
ax = sns.countplot(x='Label', data=df)
ax.set(xlabel='Attack Type', ylabel='Number of Attacks')
plt.show()

In [ ]:
df.shape

In [ ]:
gc.collect()

In [ ]:
# check for some null or missing values in our dataset
df.isna().sum().to_numpy()

In [ ]:
# Drop the rows where at least one element is missing.
cleaned_data = df.dropna()

In [ ]:
cleaned_data.shape

In [ ]:
num_removed = df.shape[0] - cleaned_data.shape[0]
print(num_removed)

In [ ]:
gc.collect()

In [ ]:
#The Label feature in the data contains 7 labels
#All these are in string format, so we need to convert them into numbers so that our NN may understand their representations.

label_encoder = LabelEncoder()
cleaned_data['Label']= label_encoder.fit_transform(cleaned_data['Label'])
cleaned_data['Label'].unique()

In [ ]:
cleaned_data['Label'].value_counts()

In [ ]:
# make 7 seperate datasets for 7 feature labels
for i in range(0,8):
    locals()['data_'+str(i)] = cleaned_data[cleaned_data['Label'] == i]
    #　print('Print In Once: ', locals()['data_'+str(i)])

print(data_0.shape[0])
print(data_1.shape[0])
print(data_2.shape[0])
print(data_3.shape[0])
print(data_4.shape[0])
print(data_5.shape[0])
print(data_6.shape[0])

In [ ]:
y_1 = np.zeros(data_0.shape[0]) # benign
y1 = pd.DataFrame(y_1)

y_2 = np.ones(data_1.shape[0])
y2 = pd.DataFrame(y_2)

y_3 = np.full(data_2.shape[0], 2)
y3 = pd.DataFrame(y_3)

y_4 = np.full(data_3.shape[0], 3)
y4 = pd.DataFrame(y_4)

y_5 = np.full(data_4.shape[0], 4)
y5 = pd.DataFrame(y_5)

y_6 = np.full(data_5.shape[0], 5)
y6 = pd.DataFrame(y_6)

y_7 = np.full(data_6.shape[0], 6)
y7 = pd.DataFrame(y_7)


X = pd.concat([data_0, data_1, data_2, data_3, data_4, data_5, data_6], sort=True)
y = pd.concat([y1, y2, y3, y4, y5, y6, y7], sort=True)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# checking if there are some null values in data
X.isnull().sum().to_numpy()

In [6]:
from sklearn.utils import resample

data_0_resample = resample(data_0, n_samples=20000, 
                           random_state=123, replace=True)
data_1_resample = resample(data_1, n_samples=20000, 
                           random_state=123, replace=True)
data_2_resample = resample(data_2, n_samples=20000,
                           random_state=123, replace=True)
data_3_resample = resample(data_3, n_samples=20000, 
                           random_state=123, replace=True)
data_4_resample = resample(data_4, n_samples=20000, 
                           random_state=123, replace=True)

NameError: ignored

In [ ]:
train_dataset = pd.concat([data_0_resample, data_1_resample, data_2_resample, data_3_resample, data_4_resample])
train_dataset.head()

In [ ]:
# viewing the distribution of intrusion attacks in our dataset 
plt.figure(figsize=(10, 8))
circle = plt.Circle((0, 0), 0.7, color='white')
plt.title('Intrusion Attack Type Distribution')
plt.pie(train_dataset['Label'].value_counts(), labels=['Benign', 'FTP-BruteForce', 'SSH-Bruteforce ', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris'],
        colors=['blue', 'gold', 'brown', 'magenta', 'cyan', 'red', 'green'])
p = plt.gcf()
p.gca().add_artist(circle)

In [ ]:
# Making X & Y Variables

test_dataset = train_dataset.sample(frac=0.1)
target_train = train_dataset['Label']
target_test = test_dataset['Label']
target_train.unique(), target_test.unique()

In [ ]:
# y_train = target_train.values.reshape(len(target_train),1)
# y_test = target_test.values.reshape(len(target_test), 1)

In [ ]:
# for CNN ,LSTM output layer dim = 5
y_train = to_categorical(target_train, num_classes=5)
y_test = to_categorical(target_test, num_classes=5)

In [ ]:
train_dataset = train_dataset.drop(columns = ["Timestamp", "Protocol","PSH Flag Cnt","Init Fwd Win Byts","Flow Byts/s","Flow Pkts/s", "Label"], axis=1)
test_dataset = test_dataset.drop(columns = ["Timestamp", "Protocol","PSH Flag Cnt","Init Fwd Win Byts","Flow Byts/s","Flow Pkts/s", "Label"], axis=1)

In [ ]:
gc.collect()

In [ ]:
# making train & test splits
X_train = train_dataset.iloc[:, :-1].values
X_test = test_dataset.iloc[:, :-1].values

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# #BernoulliNB in different binarize with 'rating'
# from sklearn.naive_bayes import BernoulliNB
# from sklearn import metrics

# BernoulliNB_accuracy = []
# for i in range(5,60,5):
#     # 建立分類器
#     model = BernoulliNB(binarize=i) # naive_bayes是計算每個feature在每筆資料中屬於正常或異常的可能性，所以要先將所有feature二值化，binarize就是設定二值化的threshold
#     Asso_clf = model.fit(X_train, y_train)

#     # 預測
#     test_y_predicted = Asso_clf.predict(X_test)
#     accuracy = metrics.accuracy_score(y_test, test_y_predicted)
#     BernoulliNB_accuracy.append(accuracy)
#     print("when binarize={}".format(i),"accuracy:{}".format(accuracy))
# plt.scatter(np.arange(5,60,5), BernoulliNB_accuracy)
# plt.xlabel("binarize")
# plt.ylabel("accuracy")
# plt.show()

In [ ]:
# # Decision Tree in different max_depth with 'rating'

# from sklearn.model_selection import cross_val_score
# from sklearn.tree import DecisionTreeClassifier
# import sklearn.tree as tree
# from sklearn import metrics

# Decision_accuracy = []
# # Decision tree
# # Max depth:The maximum depth of the tree.
# # If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
# for i in range(1,15,3):
#     # 建立分類器
#     clf = DecisionTreeClassifier(max_depth=i)
#     Asso_clf = clf.fit(X_train, y_train)

#     # 預測
#     test_y_predicted = Asso_clf.predict(X_test)
#     accuracy = metrics.accuracy_score(y_test, test_y_predicted)
#     Decision_accuracy.append(accuracy)
#     print("when max_depth={}".format(i), "accuracy:{}".format(accuracy))

# plt.scatter(np.arange(1,15,3), Decision_accuracy)
# plt.xlabel("depth")
# plt.ylabel("accuracy")
# plt.show()
# #     fig = plt.figure(figsize=(25,20))
# #     _ = tree.plot_tree(clf,filled=True)
# #     fig.savefig("decistion_tree_withdepth{}.png".format(i))

In [ ]:
# #GaussianNB without 'rating'
# from sklearn.naive_bayes import GaussianNB
# # 建立分類器
# model = GaussianNB()
# Asso_clf = model.fit(X_train, y_train)

# # 預測
# test_y_predicted = Asso_clf.predict(X_test)
# accuracy = metrics.accuracy_score(y_test, test_y_predicted)
# print("accuracy:{}".format(accuracy))

In [ ]:
# #KNN with 'rating'
# from sklearn import neighbors
# # 選擇 k
# range = np.arange(20,500,100)
# accuracies = []

# for i in range:
#     model = neighbors.KNeighborsClassifier(n_neighbors = i)
#     Asso_clf = model.fit(X_train, y_train)
#     test_y_predicted = Asso_clf.predict(X_test)
#     accuracy = metrics.accuracy_score(y_test, test_y_predicted)
#     accuracies.append(accuracy)

# # 視覺化
# plt.scatter(range, accuracies)
# plt.show()
# appr_k = accuracies.index(max(accuracies)) + 1

In [ ]:
# from sklearn import neighbors
# # 選擇 k
# range = np.arange(1,20,4)
# accuracies = []

# for i in range:
#     model = neighbors.KNeighborsClassifier(n_neighbors = i)
#     Asso_clf = model.fit(X_train, y_train)
#     test_y_predicted = Asso_clf.predict(X_test)
#     accuracy = metrics.accuracy_score(y_test, test_y_predicted)
#     accuracies.append(accuracy)

# # 視覺化
# plt.scatter(range, accuracies)
# plt.show()

# LSTM

In [ ]:
# import math
# from keras.layers import LSTM
# from keras.layers import Reshape
# from sklearn.metrics import mean_squared_error

In [ ]:
# # adam = keras.optimizers.Adam(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# def LSTM_Model():
#     model = Sequential()
#     model.add(LSTM(64, input_shape=(72, 1)))
#     model.add(Reshape((64,1)))
#     model.add(LSTM(32))
#     model.add(Reshape((32,1)))
#     model.add(LSTM(16))
#     model.add(Dense(5,activation="softmax"))
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

In [ ]:
# model_lstm = LSTM_Model()
# model_lstm.summary()

# # None代表Batch size，因為在設模型架構時沒定義，所以為None。若要先定義好則:input_shape=(10, 72, 1),
# # 代表Batch size=10，但通常在fit()內設定即可。
# # 64代表輸出的維度。因為LSTM輸入需要3維，所以要reshape，reshape後變成[64, 1, 1]

In [ ]:
# logger = [CSVLogger('logs.csv', append=True),
#           EarlyStopping(monitor='val_loss', patience=3),
#          ]
# X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
# X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],1))
# his = model_lstm.fit(X_train, y_train, epochs=30, 
#           validation_data=(X_test, y_test), callbacks=logger)

In [ ]:
# # check the model performance on test data
# scores = model_lstm.evaluate(X_test, y_test)
# print("%s: %.2f%%" % (model_lstm.metrics_names[1], scores[1] * 100))

In [ ]:
# # check history of model
# history = his.history
# history.keys()

In [ ]:
# epochs = np.arange(1, len(history['loss']) + 1)
# acc = history['accuracy']
# loss = history['loss']
# val_acc = history['val_accuracy']
# val_loss = history['val_loss']

# # visualize training and val accuracy
# plt.figure(figsize=(10, 5))
# plt.title('Training and Validation Accuracy (LSTM)')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.plot(epochs, acc, label='accuracy')
# plt.plot(epochs, val_acc, label='val_acc')
# plt.legend()

# # visualize train and val loss
# plt.figure(figsize=(10, 5))
# plt.title('Training and Validation Loss(LSTM)')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.plot(epochs, loss, label='loss', color='g')
# plt.plot(epochs, val_loss, label='val_loss', color='r')
# plt.legend()

# CNN

In [ ]:
# reshape the data for CNN
X_train = X_train.reshape(len(X_train), X_train.shape[1], 1)
X_test = X_test.reshape(len(X_test), X_test.shape[1], 1)
X_train.shape, X_test.shape

In [ ]:
# making the deep learning function
def CNN_Model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu', 
                    padding='same', input_shape=(72, 1)))
    model.add(BatchNormalization())
    
    # adding a pooling layer
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))
    
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu', 
                    padding='same', input_shape=(72, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))
    
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu', 
                    padding='same', input_shape=(72, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model_cnn = CNN_Model()
model_cnn.summary()

In [ ]:
logger = [CSVLogger('logs.csv', append=True),
          EarlyStopping(monitor='val_loss', patience=3),
         ]
his = model_cnn.fit(X_train, y_train, epochs=30, batch_size=32, 
          validation_data=(X_test, y_test), callbacks=[logger])

# lower learning rate...

In [ ]:
# check the model performance on test data
scores = model_cnn.evaluate(X_test, y_test)
print("%s: %.2f%%" % (model_cnn.metrics_names[1], scores[1] * 100))

In [ ]:
# check history of model
history = his.history
history.keys()

In [ ]:
epochs = np.arange(1, len(history['loss']) + 1)
acc = history['accuracy']
loss = history['loss']
val_acc = history['val_accuracy']
val_loss = history['val_loss']

# visualize training and val accuracy
plt.figure(figsize=(10, 5))
plt.title('Training and Validation Accuracy (CNN)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(epochs, acc, label='accuracy')
plt.plot(epochs, val_acc, label='val_acc')
plt.legend()

# visualize train and val loss
plt.figure(figsize=(10, 5))
plt.title('Training and Validation Loss(CNN)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(epochs, loss, label='loss', color='g')
plt.plot(epochs, val_loss, label='val_loss', color='r')
plt.legend()

## Save the model and weights

In [ ]:
from keras.models import load_model

model_cnn.save('CICIDS2018_model.h5')  # creates a HDF5 file 'model.h5'